In [58]:
import pandas as pd
import numpy as np
import re
import random
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFECV

In [59]:
train_file = 'cup98LRN.txt'
test_file = 'cup98VAL.txt'

train_data = pd.read_csv(train_file, delimiter=',', header=0)
test_data = pd.read_csv(test_file, delimiter=',', header=0)
train_data = train_data.fillna(train_data.mean())
test_data = test_data.fillna(test_data.mean())

X = train_data.drop(columns=['TARGET_B','TARGET_D'])
y = train_data[['TARGET_B', 'TARGET_D']]
y_b = train_data[['TARGET_B']]

In [60]:
test_data[['CONTROLN']]

,CONTROLN
0,188946
1,126296
2,155244
3,123985
4,119118
...,...
96362,51744
96363,78942
96364,82897
96365,110328


In [61]:
test_col_names = list(test_data)
test_data = test_data.sort_values(by='CONTROLN',ascending = True)
test_data = test_data.to_numpy()
test_data = pd.DataFrame(test_data, columns=test_col_names)
test_data.to_csv('./new_test.txt', index=False, header=test_col_names)
test_data = pd.read_csv('new_test.txt', delimiter=',', header=0)

C:\Users\xueyu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [62]:
test_data[['CONTROLN']]

,CONTROLN
0,3
1,6
2,9
3,11
4,16
...,...
96362,191769
96363,191771
96364,191773
96365,191774


In [63]:
y_d = train_data[[ 'TARGET_D']]

In [64]:
y_d_drop = y_d
y_d_drop = np.array(y_d_drop)
y_d_drop_list = []
for i in range(len(y_d_drop)):
    if y_d_drop[i][0] != 0:
        y_d_drop_list.append([i,y_d_drop[i][0]])

index_donation = pd.DataFrame(y_d_drop_list,columns=['index', 'donation'])
# print(index_donation)
# print("-------------------------------------------------------")
index_donation_rows = index_donation.shape[0]
top_10percent = int(0.1 * index_donation_rows)
index_donation_sort = index_donation.sort_values(by='donation',ascending = False).head(top_10percent)
# print(index_donation_sort)
# print("-------------------------------------------------------")
index = index_donation_sort['index']
index = np.array(index)
# print(index)
# print("-------------------------------------------------------")
high_value_donation = []
for i in range(y_d.shape[0]):
    if i not in index:
        high_value_donation.append([0])
    else:
        high_value_donation.append([1])
high_value_donation_pd = pd.DataFrame(high_value_donation,columns=['TARGET_D_0/1'])
high_value_donation_pd.to_csv('high_value_donation_pd.csv', index=False, header=None)
# high_value_donation_pd

In [65]:
# y_d_rows = y_d.shape[0]
# top_1percent = int(0.01 * y_d_rows)
# y_d_sort = y_d.sort_values(by='TARGET_D',ascending = False).head(top_1percent)
# y_d_sort

In [66]:
# y_b.shape

In [67]:
# oversample = RandomOverSampler()
# X_resample, y_resample = oversample.fit_resample(X, y_b)

In [68]:
oversample = RandomOverSampler()
X_resample, y_resample = oversample.fit_resample(X, y_b)
oversample1 = RandomOverSampler()
X_resample_high, y_resample_high = oversample1.fit_resample(X, high_value_donation_pd)

In [69]:
y_resample = np.reshape(y_resample.to_numpy(), y_resample.shape[0])
y_resample_high = np.reshape(y_resample_high.to_numpy(), y_resample_high.shape[0])

In [70]:
y_b.shape

(95412, 1)

In [71]:
# X_resample

In [72]:
# y_resample.describe()

In [73]:
# CONTROLN = X['CONTROLN']
# CONTROLN = CONTROLN.sort_values()
# CONTROLN.head(50)

In [74]:
# X.describe()

In [75]:
# y.describe()

In [76]:
# # people who donated
# y_resample_b1 = y_resample.loc[(y_resample['TARGET_B'] == 1.0)]

In [77]:
# y_resample_b1.describe()

In [78]:
# # People who donate more than average amount
# y_d1 = y.loc[(y['TARGET_D'] > 15)]

In [79]:
# y_d1.describe()

In [80]:
X_resample = X_resample.drop(columns=['CONTROLN'])
test_data = test_data.drop(columns=['CONTROLN'])
X_resample_high = X_resample_high.drop(columns=['CONTROLN'])

In [81]:
# test_data.head()

In [82]:
X_features = X_resample.select_dtypes(include='object')
test_features = test_data.select_dtypes(include='object')
X_features_high = X_resample_high.select_dtypes(include='object')
# features = features.to_string()
# features = features.applymap(str)
# feature_labels = list(features)
# feature_labels.remove('ZIP')

In [83]:
# features[['NOEXCH']].values

In [84]:
# X_resample[['NOEXCH']].to_numpy()

In [85]:
X_features = X_features.applymap(str)
test_features =test_features.applymap(str)

X_features_high = X_features_high.applymap(str)

In [87]:
test_features.shape

(96367, 74)

In [88]:
X_features['NOEXCH'] = X_features['NOEXCH'].replace('X', '1')
test_features['NOEXCH'] = test_features['NOEXCH'].replace('X', '1')
X_features_high['NOEXCH'] = X_features_high['NOEXCH'].replace('X', '1')
# dup_test_features = test_features

In [89]:
# X_features.head()

In [90]:
feature_labels = list(X_features)
feature_labels.remove('ZIP')

In [91]:
# np.set_printoptions(threshold=np.inf)
# features[['NOEXCH']].to_numpy()


In [92]:
# features[['NOEXCH']].to_numpy()
# count = 0
# for i in features[['NOEXCH']].to_numpy():
#     if len(i) != 1:
#         print(i)
#     else:
#         count += 1
# print(count)
# print(len(features[['NOEXCH']].to_numpy()))

In [93]:
# nominal_labels = []
# for i in range(200):
#     nominal_labels.append(feature_labels[i])

In [94]:
print(len(feature_labels))

73


In [95]:
# pd.options.display.max_rows = None
# X_resample[['NOEXCH']].values

In [96]:
# X_resample['NOEXCH'] = X_resample.to_string(X_resample['NOEXCH'])

In [97]:
# ordinal_labels = ['ODATEDW', 'ZIP',, 'DOB', '']

In [98]:
# # show all columns
# pd.options.display.max_columns = None
# X.head(20)

In [99]:
# nominal_labels = ['OSOURCE', 'STATE', 'ZIP', 'MAILCODE', 'PVASTATE', 'DOB', 'RECINHSE', 'RECP3']

In [100]:
# X_features['ZIP'].head()

In [101]:
# def categoricalHandler(train_nom, test_nom, column):
#     le = LabelEncoder()
#     labels = pd.concat([train_nom[f'{column}'], test_nom[f'{column}']], axis=0)
#     le.fit(labels)
#     train_nom[f'{column}'] = le.transform(train_nom[f'{column}'])
#     test_nom[f'{column}'] = le.transform(test_nom[f'{column}'])

#     ohe = OneHotEncoder()
#     labels = pd.concat([train_nom[[f'{column}']], test_nom[[f'{column}']]], axis=0)
#     ohe.fit(labels)
#     train_feature_arr = ohe.transform(train_nom[[f'{column}']]).toarray()
#     test_feature_arr = ohe.transform(test_nom[[f'{column}']]).toarray()
#     _feature_labels = list(le.classes_)

#     train_features = pd.DataFrame(train_feature_arr, columns=_feature_labels)
#     test_features = pd.DataFrame(test_feature_arr, columns=_feature_labels)
    
#     train_nom = pd.concat([train_nom, train_features], axis=1)
#     train_nom = train_nom.drop(columns = f'{column}')
    
#     test_nom = pd.concat([test_nom, test_features], axis=1)
#     test_nom = test_nom.drop(columns = f'{column}')

#     return train_nom, test_nom

In [102]:
def categoricalHandler(train_nom, X_high, test_nom, column):
    le = LabelEncoder()
    labels = pd.concat([train_nom[f'{column}'], test_nom[f'{column}']], axis=0)
    le.fit(labels)
    train_nom[f'{column}'] = le.transform(train_nom[f'{column}'])
    test_nom[f'{column}'] = le.transform(test_nom[f'{column}'])
    X_high[f'{column}'] = le.transform(X_high[f'{column}']) #

    ohe = OneHotEncoder()
    labels = pd.concat([train_nom[[f'{column}']], test_nom[[f'{column}']]], axis=0)
    ohe.fit(labels)
    train_feature_arr = ohe.transform(train_nom[[f'{column}']]).toarray()
    X_high_arr = ohe.transform(X_high[[f'{column}']]).toarray()
    test_feature_arr = ohe.transform(test_nom[[f'{column}']]).toarray() #
    _feature_labels = list(le.classes_)

    train_features = pd.DataFrame(train_feature_arr, columns=_feature_labels)
    test_features = pd.DataFrame(test_feature_arr, columns=_feature_labels)
    high_features = pd.DataFrame(X_high_arr, columns=_feature_labels) #
    
    train_nom = pd.concat([train_nom, train_features], axis=1)
    train_nom = train_nom.drop(columns = f'{column}')
    
    test_nom = pd.concat([test_nom, test_features], axis=1)
    test_nom = test_nom.drop(columns = f'{column}')
    
    X_high = pd.concat([X_high, high_features], axis=1)
    X_high = X_high.drop(columns = f'{column}')

    return train_nom, X_high, test_nom

In [103]:
for label in feature_labels:
    X_features, X_features_high, test_features = categoricalHandler(X_features, X_features_high, test_features, label)

In [104]:
# X_features_high.head()

In [105]:
# X_features.head()

In [106]:
X_ZIP = X_features[['ZIP']]
X_features = X_features.drop(columns=['ZIP'])
test_ZIP = test_features[['ZIP']]
test_features = test_features.drop(columns=['ZIP'])
X_ZIP_high = X_features_high[['ZIP']]
X_features_high = X_features_high.drop(columns=['ZIP'])

In [107]:
X_arr = np.array(X_features)
test_arr = np.array(test_features)
X_arr_high = np.array(X_features_high)

In [108]:
def dimensionReduce(arr):
    pca = PCA(n_components = 0.9)
    pca.fit(arr)
    _arr = pca.transform(arr)
    return _arr

In [109]:
# X_arr = pd.DataFrame(X_arr)
# X_arr_fillna = X_arr.fillna(X_arr.mean())
# test_arr = pd.DataFrame(test_arr)
# test_arr_fillna = test_arr.fillna(test_arr.mean())

# X_arr_high = pd.DataFrame(X_arr_high)
# X_arr_high_fillna = X_arr_high.fillna(X_arr_high.mean())

In [110]:
# rfc1 = RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=2, random_state=42)
# selector1 = RFECV(estimator= rfc1, step=1, cv=5, scoring = 'f1')
# X_arr = selector1.fit_transform(X_arr, y_resample)
# test_arr = selector1.transform(test_arr)

In [111]:
# rfc2 = RandomForestClassifier(max_depth=4, min_samples_leaf=1, min_samples_split=2, random_state=42)
# selector2 = RFECV(estimator= rfc2, step=1, cv=5, scoring = 'f1')
# X_arr_high = selector2.fit_transform(X_arr_high, y_resample_high)

In [112]:
X_arr = dimensionReduce(X_arr)
test_arr = dimensionReduce(test_arr)
X_arr_high = dimensionReduce(X_arr_high)

In [113]:
X_features = pd.DataFrame(X_arr)
test_features = pd.DataFrame(test_arr)
X_features_high = pd.DataFrame(X_arr_high)

In [114]:
def cleanZip(zip):
    z_arr = np.array(zip)
    for z in z_arr:
        z[0] = re.sub('-', '', z[0])
    _zip = pd.DataFrame(z_arr, columns=['ZIP'])
    return _zip

In [115]:
X_ZIP = cleanZip(X_ZIP)
test_ZIP = cleanZip(test_ZIP)
X_ZIP_high = cleanZip(X_ZIP_high)

In [116]:
X_ZIP = np.array(X_ZIP)
test_ZIP = np.array(test_ZIP)
X_ZIP = np.reshape(X_ZIP, len(X_ZIP))
test_ZIP = np.reshape(test_ZIP, len(test_ZIP))
X_ZIP = pd.to_numeric(X_ZIP)
test_ZIP = pd.to_numeric(test_ZIP)
X_ZIP = np.reshape(X_ZIP, (len(X_ZIP), 1))
test_ZIP = np.reshape(test_ZIP, (len(test_ZIP), 1))
X_ZIP = pd.DataFrame(X_ZIP)
test_ZIP = pd.DataFrame(test_ZIP)

X_ZIP_high = np.array(X_ZIP_high)
X_ZIP_high = np.reshape(X_ZIP_high, len(X_ZIP_high))
X_ZIP_high = pd.to_numeric(X_ZIP_high)
X_ZIP_high = np.reshape(X_ZIP_high, (len(X_ZIP_high), 1))
X_ZIP_high = pd.DataFrame(X_ZIP_high)

In [117]:
X_ZIP.shape

(181138, 1)

In [118]:
X_features = pd.concat([X_ZIP, X_features], axis=1)
test_features = pd.concat([test_ZIP, test_features], axis=1)
X_features_high = pd.concat([X_ZIP_high, X_features_high], axis=1)

In [119]:
X_features.shape

(181138, 413)

In [120]:
feature_labels.append('ZIP')
X_resample = X_resample.drop(columns=feature_labels)
test_data = test_data.drop(columns=feature_labels)
X_resample = pd.concat([X_resample, X_features], axis=1)
test_data = pd.concat([test_data, test_features], axis=1)

X_resample_high = X_resample_high.drop(columns=feature_labels)
X_resample_high = pd.concat([X_resample_high, X_features_high], axis=1)

In [121]:
X_resample.shape

(181138, 817)

In [122]:
X_arr = np.array(X_resample)
test_arr = np.array(test_data)
X_arr_high = np.array(X_resample_high)

In [123]:
# X_arr = pd.DataFrame(X_arr)
# X_arr_fillna = X_arr.fillna(X_arr.mean())
# test_arr = pd.DataFrame(test_arr)
# test_arr_fillna = test_arr.fillna(test_arr.mean())

# X_arr_high = pd.DataFrame(X_arr_high)
# X_arr_high_fillna = X_arr_high.fillna(X_arr_high.mean())

In [124]:
# X_arr = selector1.fit_transform(X_arr, y_resample)
# test_arr = selector1.transform(test_arr)
# X_arr_high = selector2.fit_transform(X_arr_high, y_resample_high)

In [125]:
# clean NaN values before continuing
X_arr = dimensionReduce(X_arr)
test_arr = dimensionReduce(test_arr)
X_arr_high = dimensionReduce(X_arr_high)

In [126]:
X_resample = pd.DataFrame(X_arr)
test_data = pd.DataFrame(test_arr)
X_resample_high = pd.DataFrame(X_arr_high)

In [127]:
# X_resample.head()

In [128]:
# X_resample['ZIP'].head()

In [129]:
X_resample.shape

(181138, 2)

In [130]:
# test_data.head()

In [131]:
test_data.shape

(96367, 2)

In [132]:
# y_resample = np.reshape(y_resample.to_numpy(), y_resample.shape[0])
# y_resample_high = np.reshape(y_resample_high.to_numpy(), y_resample_high.shape[0])

In [133]:
y_resample.shape

(181138,)

In [134]:
# X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size = 0.2, random_state=42)

In [135]:
# RF = RandomForestClassifier(random_state=42)
# tune_array_1_10 = list(range(1,11))
# tune_array_2_5 = list(range(2,6))
# tune_array_1_10 = list(range(1,11))
# # use GridSearch to do hypeparameter tuning in order to get the best model
# parameters = { 'max_depth':tune_array_1_10,'min_samples_leaf':tune_array_1_10,'min_samples_split':tune_array_2_5  }

# clf = GridSearchCV(estimator = RF, param_grid = parameters, scoring = 'f1', cv = 5, return_train_score = True, n_jobs=-1)
# clf.fit(X_train, y_train)
# print("best parameters are ", clf.best_params_)
# label_predict = clf.predict(X_test)
# print("length of label_predict:",len(label_predict))
# print("f1 score (average = None) :", f1_score(y_test, label_predict, average=None))
# print("f1 score (average = macro) :", f1_score(y_test, label_predict, average='macro'))
# print("accuracy_score:", accuracy_score(y_test, label_predict))
# print("confusion_matrix:\n", confusion_matrix(y_test, label_predict))

In [136]:
rfc1 = RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=2, random_state=42)
# rfc = DecisionTreeClassifier(random_state=42)

In [137]:
score = cross_validate(rfc1, X_resample, y_resample, scoring = 'f1')
score

{'fit_time': array([11.05424619, 10.78012872, 10.928298  , 10.81279278, 10.8372488 ]),
 'score_time': array([0.20649314, 0.20984054, 0.19801116, 0.19865584, 0.19010448]),
 'test_score': array([0.65456391, 0.62915352, 0.63167509, 0.63750426, 0.64299563])}

In [138]:
test_data.shape

(96367, 2)

In [140]:
# test_data = test_data.drop(columns=2)

In [141]:
rfc1.fit(X_resample, y_resample)
predictions_all = rfc1.predict(test_data)

In [142]:
# X_train, X_test, y_train, y_test = train_test_split(X_resample_high, y_resample_high, test_size = 0.2, random_state=42)
# RF = RandomForestClassifier(random_state=42)
# tune_array_1_10 = list(range(1,11))
# tune_array_2_5 = list(range(2,5))
# tune_array_1_10 = list(range(1,5))
# # use GridSearch to do hypeparameter tuning in order to get the best model
# parameters = { 'max_depth':tune_array_1_10,'min_samples_leaf':tune_array_1_10,'min_samples_split':tune_array_2_5  }

# clf = GridSearchCV(estimator = RF, param_grid = parameters, scoring = 'f1', cv = 5, return_train_score = True, n_jobs=-1)
# clf.fit(X_train, y_train)
# print("best parameters are ", clf.best_params_)
# label_predict = clf.predict(X_test)
# print("length of label_predict:",len(label_predict))
# print("f1 score (average = None) :", f1_score(y_test, label_predict, average=None))
# print("f1 score (average = macro) :", f1_score(y_test, label_predict, average='macro'))
# print("accuracy_score:", accuracy_score(y_test, label_predict))
# print("confusion_matrix:\n", confusion_matrix(y_test, label_predict))

In [143]:
rfc2 = RandomForestClassifier(max_depth=4, min_samples_leaf=1, min_samples_split=2, random_state=42)
score = cross_validate(rfc2, X_resample_high, y_resample_high, scoring = 'f1')
score

{'fit_time': array([4.90511513, 5.10585165, 4.94105673, 5.03401041, 4.91491485]),
 'score_time': array([0.14803386, 0.15903616, 0.14903641, 0.15003419, 0.15403557]),
 'test_score': array([0.6312698 , 0.630361  , 0.63174279, 0.62941958, 0.64044431])}

In [144]:
rfc2.fit(X_resample_high, y_resample_high)
predictions_high = rfc2.predict(test_data)

In [234]:
count_both = 0
count_high = 0
count_all = 0
predictions_final = []
prob = 0.8
for i in range(len(predictions_all)):
    if predictions_all[i]==1:
        predictions_final.append('1')
        count_both += 1
#     if prz
#             count_high += 1
#     elif predictions_all[i]==1 and random.random()>=prob:
#             predictions_final.append('1')
#             count_all += 1
    else:
        predictions_final.append('0')

In [235]:
print("count_both", count_both)

count_both 32469


In [236]:
print("count_high", count_high)

count_high 0


In [237]:
print("count_all", count_all)

count_all 0


In [238]:
# predictions_final

In [239]:
predictions_final = pd.DataFrame(predictions_final,columns=['label'])

In [240]:
value_file = 'valtargt.txt'
value_df = pd.read_csv(value_file, delimiter=',', header=0)

In [241]:
value_df = pd.concat([value_df, predictions_final], axis=1)

In [242]:
donor_df = value_df.loc[(value_df['label']=='1')]
amount_donated = donor_df['TARGET_D'].sum()
cost = donor_df.shape[0] * 0.68
revenue = amount_donated - cost

In [243]:
max_amount_donated = value_df['TARGET_D'].sum()
max_cost = value_df['TARGET_B'].sum() * 0.68
max_revenue = max_amount_donated - max_cost

In [244]:
ratio = revenue/max_revenue

In [245]:
amount_donated

28696.0

In [246]:
revenue

6617.079999999998

In [247]:
cost

22078.920000000002

In [219]:
max_cost

3313.6400000000003